In [1]:
from pkgutil import read_code

from mlcroissant import RecordSet

from cryoet_data_portal_croissant.gen import generate_mlcroissant
import mlcroissant as mlc
import json

ds = generate_mlcroissant(dataset_ids=[10000])

with open("test.json", "w") as f:
    f.write(json.dumps(ds[0].to_json(), indent=4, default=str) + '\n')

Found the following 2 warning(s) during the validation:
  -  [Metadata(S. pombe cells with defocus)] Property "http://mlcommons.org/croissant/citeAs" is recommended, but does not exist.
  -  [Metadata(S. pombe cells with defocus)] Property "https://schema.org/version" is recommended, but does not exist.


In [3]:
dataset = mlc.Dataset.from_metadata(ds[0])
dataset.debug = True
records = dataset.records(record_set="runs")

for record in records:
    print(record)


  -  [Metadata(S. pombe cells with defocus)] Property "http://mlcommons.org/croissant/citeAs" is recommended, but does not exist.
  -  [Metadata(S. pombe cells with defocus)] Property "https://schema.org/version" is recommended, but does not exist.


{'runs/id': 249, 'runs/name': '10000/TS_043'}
{'runs/id': 241, 'runs/name': '10000/TS_026'}
{'runs/id': 242, 'runs/name': '10000/TS_027'}
{'runs/id': 243, 'runs/name': '10000/TS_028'}
{'runs/id': 244, 'runs/name': '10000/TS_029'}
{'runs/id': 245, 'runs/name': '10000/TS_030'}
{'runs/id': 246, 'runs/name': '10000/TS_034'}
{'runs/id': 247, 'runs/name': '10000/TS_037'}
{'runs/id': 248, 'runs/name': '10000/TS_041'}
{'runs/id': 250, 'runs/name': '10000/TS_045'}


In [19]:
from linkml_runtime.utils.schemaview import SchemaView

sv = SchemaView('schema.yaml')

print(dir(sv.all_classes()["Run"]))

sv.all_classes()["Run"].attributes['tiltseries']

['MissingRequiredField', '__annotations__', '__bool__', '__class__', '__contains__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__post_init__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_as_dict', '_as_json', '_as_json_dumps', '_as_json_obj', '_class_for', '_class_for_curie', '_class_for_uri', '_default', '_get', '_hide_list', '_idempotent', '_if_missing', '_inherited_slots', '_init_from_dict', '_is_empty', '_items', '_keys', '_normalize_inlined', '_normalize_inlined_as_dict', '_normalize_inlined_as_list', '_normalize_inlined_slot', '_setdefault', '_static_default

SlotDefinition({
  'name': 'tiltseries',
  'annotations': {'cascade_delete': Annotation(tag='cascade_delete',
                                 value=True,
                                 extensions={},
                                 annotations={})},
  'from_schema': 'cdp-dataset-config',
  'inverse': 'Tiltseries.run',
  'range': 'Tiltseries',
  'multivalued': True
})

In [19]:
TYPE_MAP = {
    "string": mlc.DataType.TEXT,
    "int": mlc.DataType.INTEGER,
    "float": mlc.DataType.FLOAT,
    "boolean": mlc.DataType.BOOL,
    "date": mlc.DataType.DATE,
    "datetime": mlc.DataType.DATE,
    "url": mlc.DataType.URL,
}

import re
def camel_to_snake(name):
    # Add an underscore before each capital letter and convert to lowercase
    return re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()

def class_to_recordset(linkml_class):
    """
    Convert a class to a RecordSet.

    Args:
        sv: SchemaView object.
        cls: Class to convert.

    Returns:
        mlc.RecordSet: The converted RecordSet.
    """

    fields = []

    clsname = camel_to_snake(linkml_class.name)

    for attr, slot_def in linkml_class.attributes.items():
        if slot_def.range in TYPE_MAP:
            fld = mlc.Field(
                id = f"{clsname}/{slot_def.name}",
                data_types = [TYPE_MAP[slot_def.range]],
                name = slot_def.name,
                description = slot_def.description,

            )
        else:
            fld = mlc.Field(
                id = f"{clsname}/{slot_def.name}",
                name = slot_def.name,
                description = slot_def.description,
                references=mlc.Source(id=f"{slot_def.range}/id"),
            )

        fields.append(fld)

    rs = mlc.RecordSet(
        id = f"{clsname}",
        name = f"{clsname}",
        fields = fields,
    )

    return rs

class_to_recordset(sv.all_classes()["Run"]).to_json()

{'@type': 'cr:RecordSet',
 '@id': 'Run',
 'name': 'Run',
 'field': [{'@type': 'cr:Field',
   '@id': 'Run/alignments',
   'name': 'alignments',
   'description': 'Tiltseries Alignment',
   'references': {'@id': 'Alignment/id'}},
  {'@type': 'cr:Field',
   '@id': 'Run/annotations',
   'name': 'annotations',
   'description': 'Metadata about an annotation for a run',
   'references': {'@id': 'Annotation/id'}},
  {'@type': 'cr:Field',
   '@id': 'Run/dataset',
   'name': 'dataset',
   'description': 'Dataset that this run is a part of',
   'references': {'@id': 'Dataset/id'}},
  {'@type': 'cr:Field',
   '@id': 'Run/frames',
   'name': 'frames',
   'references': {'@id': 'Frame/id'}},
  {'@type': 'cr:Field',
   '@id': 'Run/gain_files',
   'name': 'gain_files',
   'references': {'@id': 'GainFile/id'}},
  {'@type': 'cr:Field',
   '@id': 'Run/frame_acquisition_files',
   'name': 'frame_acquisition_files',
   'references': {'@id': 'FrameAcquisitionFile/id'}},
  {'@type': 'cr:Field',
   '@id': 'Ru

In [ ]:
Field = mlc.Field(
    id="field",
    name="field",
    description="field",
    data_types=[mlc.DataType.TEXT],
    array_shape="(-1,)",
)

In [ ]:
obj = mlc.FileObject(
    id="id",
    name="name",
    description="description",
    content_url="https://example.com",
    encoding_formats=["image/OME-Zarr"],
    same_as=["same_as"],
    contained_in=["contained_in"],
)
obj.

In [15]:
import cryoet_data_portal as cdp
import hashlib

client = cdp.Client()
tomos = cdp.Tomogram.find(client, [cdp.Tomogram.run.dataset_id == 10000])
jtomos = [tomo.to_dict() for tomo in tomos]

with open("tomos.json", "w") as f:
    f.write(json.dumps(jtomos, indent=4, default=str) + '\n')

with open("tomos.json", "rb") as f:
    csum = hashlib.file_digest(f, hashlib.sha256)

print(csum.hexdigest())


cdp.Tomogram.

234b14620d93934980f6131d41251eec918e86b07738434b1554867c506b8077


In [16]:
obj = mlc.FileObject(
    id="tomogramsfile",
    name="name",
    description="description",
    content_url="http://0.0.0.0:8000/tomos.json",
    encoding_formats=["application/json"],
    sha256 = "234b14620d93934980f6131d41251eec918e86b07738434b1554867c506b8077",
)

rs = mlc.RecordSet(
    id = "tomograms",
    name="tomograms",
    description="Tomograms",
    key=["tomograms/id"],
    fields=[
        mlc.Field(
            id="tomograms/id",
            data_types=[mlc.DataType.INTEGER],
            description="ID of the tomogram.",
            source=mlc.Source(file_object="tomogramsfile", extract=mlc.Extract(json_path="$[*].id")),
        ),
        mlc.Field(
            id="tomograms/name",
            data_types=[mlc.DataType.TEXT],
            description="Name of the tomogram.",
            source=mlc.Source(file_object="tomogramsfile", extract=mlc.Extract(json_path="$[*].s3_omezarr_dir")),
        ),
    ],
)

md = mlc.Metadata(
    name="test",
    description="test",
    creators=[mlc.Person(name="Test Person")],
    license=["https://creativecommons.org/public-domain/cc0/"],
    url="https://example.com",
    distribution=[obj],
    record_sets=[rs],
)

ds = mlc.Dataset.from_metadata(md)

for record in ds.records(record_set="tomograms"):
    print(record)


  -  [Metadata(test)] Property "http://mlcommons.org/croissant/citeAs" is recommended, but does not exist.
  -  [Metadata(test)] Property "https://schema.org/datePublished" is recommended, but does not exist.
  -  [Metadata(test)] Property "https://schema.org/version" is recommended, but does not exist.


{'tomograms/id': 629, 'tomograms/name': b's3://cryoet-data-portal-public/10000/TS_043/Reconstructions/VoxelSpacing13.480/Tomograms/100/TS_043.zarr'}
{'tomograms/id': 621, 'tomograms/name': b's3://cryoet-data-portal-public/10000/TS_026/Reconstructions/VoxelSpacing13.480/Tomograms/100/TS_026.zarr'}
{'tomograms/id': 622, 'tomograms/name': b's3://cryoet-data-portal-public/10000/TS_027/Reconstructions/VoxelSpacing13.480/Tomograms/100/TS_027.zarr'}
{'tomograms/id': 623, 'tomograms/name': b's3://cryoet-data-portal-public/10000/TS_028/Reconstructions/VoxelSpacing13.480/Tomograms/100/TS_028.zarr'}
{'tomograms/id': 624, 'tomograms/name': b's3://cryoet-data-portal-public/10000/TS_029/Reconstructions/VoxelSpacing13.480/Tomograms/100/TS_029.zarr'}
{'tomograms/id': 625, 'tomograms/name': b's3://cryoet-data-portal-public/10000/TS_030/Reconstructions/VoxelSpacing13.480/Tomograms/100/TS_030.zarr'}
{'tomograms/id': 626, 'tomograms/name': b's3://cryoet-data-portal-public/10000/TS_034/Reconstructions/Voxe

In [35]:
cdp.Dataset.__doc__

'A collection of imaging experiments on the same organism\n\n    Attributes:\n        id (int): An identifier for a CryoET dataset, assigned by the Data Portal. Used to identify the dataset as the directory name in data tree\n        deposition (Deposition): The deposition this dataset is a part of\n        deposition_id (int): None\n        funding_sources (List[DatasetFunding]): The dataset fundings of this dataset\n        authors (List[DatasetAuthor]): The dataset authors of this dataset\n        runs (List[Run]): The runs of this dataset\n        title (str): Title of a CryoET dataset\n        description (str): A short description of a CryoET dataset, similar to an abstract for a journal article or dataset.\n        organism_name (str): Name of the organism from which a biological sample used in a CryoET study is derived from, e.g. homo sapiens\n        organism_taxid (int): NCBI taxonomy identifier for the organism, e.g. 9606\n        tissue_name (str): Name of the tissue from w

In [84]:
from griffe import Docstring
from typing import Type
from pydantic import BaseModel, create_model

_PORTAL_TYPES = (
      Type[cdp.Annotation]
    | Type[cdp.AnnotationShape]
    | Type[cdp.AnnotationFile]
    | Type[cdp.Tomogram]
    | Type[cdp.TiltSeries]
    | Type[cdp.Dataset]
    | Type[cdp.Run]
    | Type[cdp.Alignment]
)

_TYPE_MAP = {
    "str": mlc.DataType.TEXT,
    "int": mlc.DataType.INTEGER,
    "float": mlc.DataType.FLOAT,
    "bool": mlc.DataType.BOOL,
    "date": mlc.DataType.DATE,
    "datetime": mlc.DataType.DATE,
    "url": mlc.DataType.URL,
}

def get_descriptions(cls: Type[cdp.Dataset]) -> tuple[str, dict[str, str]]:
    """
    Get the description and attributes of a class from its google docstring.

    Args:
        cls: The class to extract the docstring from.
    Returns:
        tuple: A tuple containing the description and a dictionary of attributes and their descriptions.
    """
    doc = Docstring(cls.__doc__)
    attrs = doc.parse("google")
    return attrs[0].value, {a.name: a.description for a in attrs[1].value}

def _portal_to_recordset(clz: _PORTAL_TYPES) -> mlc.RecordSet:
    """Automatically create a Pydantic model from a CryoET Data Portal annotation class."""
    attrs = clz.__annotations__
    docs = get_descriptions(clz)
    fields = []
    clz_name = clz._gql_type.lower()
    filename = f"{clz._gql_root_field}.json"

    for name, typ in attrs.items():
        if name == "neuroglancer_config":
            continue
        if typ in ["int", "float", "str", "bool"] and name[0] != "_":
            data_type = _TYPE_MAP[typ]
            if typ == "str" and ("http" in name or "s3://" in name):
                data_type = mlc.DataType.URL

            if "id" in name and name != "id":
                fld = mlc.Field(
                    id=f"{clz_name}_{name}",
                    name=name,
                    data_types=[data_type],
                    description=docs[1].get(name, None),
                    source=mlc.Source(
                        file_object=filename,
                        extract=mlc.Extract(
                            json_path=f"$[*].{name}"
                        ),
                    ),
                    references=mlc.Source(id=f"{name}"),
                )
            else:
                fld = mlc.Field(
                    id=f"{clz_name}_{name}",
                    name=name,
                    data_types=[data_type],
                    description=docs[1].get(name, None),
                    source=mlc.Source(
                        file_object=filename,
                        extract=mlc.Extract(
                            json_path=f"$[*].{name}"
                        ),
                    ),
                )

            fields.append(fld)

    rs = mlc.RecordSet(
        id=f"{clz_name}",
        name=f"{clz_name}",
        description=docs[0],
        fields=fields,
        key=[f"{clz_name}_id"],
    )
    return rs

rss = _portal_to_recordset(cdp.Dataset)

rss.to_json()

{'@type': 'cr:RecordSet',
 '@id': 'dataset',
 'name': 'dataset',
 'description': 'A collection of imaging experiments on the same organism',
 'key': {'@id': 'dataset_id'},
 'field': [{'@type': 'cr:Field',
   '@id': 'dataset_id',
   'name': 'id',
   'description': 'An identifier for a CryoET dataset, assigned by the Data Portal. Used to identify the dataset as the directory name in data tree',
   'dataType': 'sc:Integer',
   'source': {'fileObject': {'@id': 'datasets.json'},
    'extract': {'jsonPath': '$[*].id'}}},
  {'@type': 'cr:Field',
   '@id': 'dataset_deposition_id',
   'name': 'deposition_id',
   'description': 'None',
   'dataType': 'sc:Integer',
   'references': {'@id': 'deposition_id'},
   'source': {'fileObject': {'@id': 'datasets.json'},
    'extract': {'jsonPath': '$[*].deposition_id'}}},
  {'@type': 'cr:Field',
   '@id': 'dataset_title',
   'name': 'title',
   'description': 'Title of a CryoET dataset',
   'dataType': 'sc:Text',
   'source': {'fileObject': {'@id': 'datase

In [85]:
import cryoet_data_portal as cdp
import hashlib

def _dump_portal(dataset_id: int) -> tuple[mlc.FileObject, mlc.RecordSet]:
    """
    Dump a CryoET Data Portal class to a list of FileObjects.

    Args:
        clz: The class to dump.

    Returns:
        list[mlc.FileObject]: A list of FileObjects.
    """

    to_dump = [
        {
            "class": cdp.Annotation,
            "query": [cdp.Annotation.run.dataset_id == dataset_id]
        },
        {
            "class": cdp.AnnotationShape,
            "query": [cdp.AnnotationShape.annotation.run.dataset_id == dataset_id]
        },
        {
            "class": cdp.AnnotationFile,
            "query": [cdp.AnnotationFile.tomogram_voxel_spacing.run.dataset_id == dataset_id]
        },
        {
            "class": cdp.Tomogram,
            "query": [cdp.Tomogram.run.dataset_id == dataset_id]
        },
        {
            "class": cdp.TiltSeries,
            "query": [cdp.TiltSeries.run.dataset_id == dataset_id]
        },
        {
            "class": cdp.Dataset,
            "query": [cdp.Dataset.id == dataset_id]
        },
        {
            "class": cdp.Run,
            "query": [cdp.Run.dataset_id == dataset_id]
        },
        {
            "class": cdp.Alignment,
            "query": [cdp.Alignment.run.dataset_id == dataset_id]
        },
    ]

    objects = []
    recordsets = []

    for dump in to_dump:
        clz = dump["class"]
        query = dump["query"]
        client = cdp.Client()
        items = clz.find(client, query)
        jitems = [item.to_dict() for item in items]
        filename = f"{clz._gql_root_field}.json"
        text = json.dumps(jitems, indent=4, default=str) + '\n'

        with open(filename, "w") as f:
            f.write(text)

        with open(filename, "rb") as f:
            checksum = hashlib.file_digest(f, hashlib.sha256).hexdigest()

        print(checksum)

        obj = mlc.FileObject(
            id=filename,
            name=filename,
            description="description",
            content_url=f"http://0.0.0.0:8000/{clz._gql_root_field}.json",
            encoding_formats=["application/json"],
            sha256 = checksum,
        )

        objects.append(obj)

        recordsets.append(_portal_to_recordset(clz))
        print(json.dumps(recordsets[-1].to_json(), indent=4, default=str))
    return objects, recordsets

objs, recordsets = _dump_portal(10000)
#rs = _portal_to_recordset(cdp.Dataset)



md = mlc.Metadata(
    name="test",
    description="test",
    creators=[mlc.Person(name="Test Person")],
    license=["https://creativecommons.org/public-domain/cc0/"],
    url="https://example.com",
    distribution=objs,
    record_sets=recordsets,
)



ds = mlc.Dataset.from_metadata(md)

# for record in ds.records(record_set="tomogram"):
#     print(record)

print(list(ds.records(record_set="tomogram"))[0])

e4392e6bbe54bb52dafa0e71a53a90b59fcde7ad28d934d308909bdbaf6bec16
{
    "@type": "cr:RecordSet",
    "@id": "annotation",
    "name": "annotation",
    "description": "Metadata for an annotation",
    "key": {
        "@id": "annotation_id"
    },
    "field": [
        {
            "@type": "cr:Field",
            "@id": "annotation_id",
            "name": "id",
            "description": "Numeric identifier (May change!)",
            "dataType": "sc:Integer",
            "source": {
                "fileObject": {
                    "@id": "annotations.json"
                },
                "extract": {
                    "jsonPath": "$[*].id"
                }
            }
        },
        {
            "@type": "cr:Field",
            "@id": "annotation_run_id",
            "name": "run_id",
            "description": "None",
            "dataType": "sc:Integer",
            "references": {
                "@id": "run_id"
            },
            "source": {
            

  -  [Metadata(test)] Property "http://mlcommons.org/croissant/citeAs" is recommended, but does not exist.
  -  [Metadata(test)] Property "https://schema.org/datePublished" is recommended, but does not exist.
  -  [Metadata(test)] Property "https://schema.org/version" is recommended, but does not exist.


5abae47c975e6b81e2562fe076231f497f3d2e2e0748b58aea526dacce93e6c3
{
    "@type": "cr:RecordSet",
    "@id": "alignment",
    "name": "alignment",
    "description": "Tiltseries Alignment",
    "key": {
        "@id": "alignment_id"
    },
    "field": [
        {
            "@type": "cr:Field",
            "@id": "alignment_id",
            "name": "id",
            "description": "Numeric identifier (May change!)",
            "dataType": "sc:Integer",
            "source": {
                "fileObject": {
                    "@id": "alignments.json"
                },
                "extract": {
                    "jsonPath": "$[*].id"
                }
            }
        },
        {
            "@type": "cr:Field",
            "@id": "alignment_deposition_id",
            "name": "deposition_id",
            "description": "None",
            "dataType": "sc:Integer",
            "references": {
                "@id": "deposition_id"
            },
            "source": {
   

In [76]:
objs

[FileObject(uuid="testdata/annotations.json"),
 FileObject(uuid="testdata/annotationShapes.json"),
 FileObject(uuid="testdata/annotationFiles.json"),
 FileObject(uuid="testdata/tomograms.json"),
 FileObject(uuid="testdata/tiltseries.json"),
 FileObject(uuid="testdata/datasets.json"),
 FileObject(uuid="testdata/runs.json"),
 FileObject(uuid="testdata/alignments.json")]